## Import

In [1]:
import numpy as np
import tensorflow as tf

from rbm.learning.adam import Adam
from rbm.rbmcf import RBMCF
from rbm.sampling.contrastive_divergence import ContrastiveDivergence

total_of_recommendations = 100

fold = 0
rating_size = 117
total_movies = 6
hidden_size = 10000
batch_size = 10

roll_outs = 10000


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



## Original random data

In [2]:
from rbm.util.embedding import one_hot_encoding

random_plugins = np.random.randint(low=0, high=rating_size, size=[total_of_recommendations, total_movies])
print('Random path plugins', random_plugins, sep='\n')

random_patches_one_hot = one_hot_encoding(random_plugins, depth=rating_size).T
print()
print('Shape', random_patches_one_hot.shape)
print('Path as one hot')
random_patches_one_hot

Random path plugins
[[ 62  26   5  89 109  56]
 [ 96  86 113  37  42  78]
 [ 53  93  47 104  85 113]
 [ 48  86  88  76   9  55]
 [ 55  81 105  50  35  51]
 [110  49  90  68  62  26]
 [113  91  88  49  15  80]
 [ 30  42  64  68  14  28]
 [ 11  95  91 106  82 100]
 [  8  67 108  43  22  35]
 [ 21  43  10  70  72  97]
 [ 30 111 113  56 109  80]
 [ 21  36  48  62  28  60]
 [ 47  51  33  35  96  93]
 [ 14  56   8  42  19  54]
 [ 70  74 110  50  92  79]
 [ 21  61  36 103  38   0]
 [ 96  95  52  71  58 109]
 [  4  19   7  43  47  76]
 [  7  73  94   2  22  58]
 [ 31  34  74  68  82  67]
 [115  80  82   8   0  48]
 [  3 110  95  38  16  83]
 [ 88  77  95  39  68  89]
 [ 79  26  68  86  74  19]
 [  2  89  33 100  37  28]
 [ 58 102  43  81  89  11]
 [ 72  35   9  78  60   2]
 [ 80 106  92  48  97  56]
 [  8 108 106  22  57   4]
 [  0  44  92  96  42   6]
 [ 74  67  45  51  21  89]
 [ 54  27 114  77  95  41]
 [ 77  94  28   7  54  93]
 [ 64 114  48  82  65 112]
 [104 105  11  62  89  74]
 [103  4

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

## Generate recommendations

In [3]:
tf.reset_default_graph()


with tf.Session() as session:
    model = RBMCF(total_movies, rating_size, hidden_size=hidden_size, sampling_method=ContrastiveDivergence(1), learning_rate=Adam(0.05), momentum=0)

    model_path = f"../results/model/kfold={fold}+kfold-intern=0+batch_size={batch_size}+{model.__str__().replace('/', '+')}/rbm.ckpt"

    model.load(session, model_path)
    
    i = tf.constant(0)
    patches = tf.constant(random_patches_one_hot)

    c = lambda i, patches: tf.less(i, roll_outs)
    b = lambda i, patches: (i+1, model.gibbs_step(patches))

    loop = tf.while_loop(c, b, [i, patches], parallel_iterations=1, back_prop=False)
    
    #for i in range(roll_outs):
    #    random_patches_one_hot = model.gibbs_step(random_patches_one_hot)
    
    generated_patches = session.run(loop)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from ../results/model/kfold=0+kfold-intern=0+batch_size=10+class=RBMCF+visible_size=702+hidden_size=10000+regularization=NoRegularization-0.0+learning_rate=Adam-0.05-0.9-0.999-1e-08+sampling_method=CD-1+momentum=0/rbm.ckpt


## Recommendations as ids (not one-hot)

In [11]:
ids = generated_patches[1].T.reshape([total_of_recommendations, total_movies, -1]).argmax(axis=2)
ids

array([[ 27,  92,   0,   0,  44,   0],
       [  0,  27,   0,   0, 107, 107],
       [ 27,   0,  30,  42,   0,  60],
       [ 24,  16,  64, 107, 107, 107],
       [  0, 103,  62,   0, 107, 107],
       [ 23,   0,   0, 107, 107, 107],
       [ 31,   0,   8,   0, 107, 107],
       [ 28,  31,   0,   0, 107, 107],
       [ 27,   0,   0,  38,  98,  71],
       [  0, 101,  62,   0, 107, 107],
       [ 27,  23,   0,  27, 107, 107],
       [  2,  90,  30,   0,  54,   0],
       [ 27,   0,   0,   0,   0,   0],
       [ 27,  23,   8,   0,  30,  17],
       [ 27,  73,  30,   0,   0,   7],
       [ 73,  30,  27, 107, 107, 107],
       [ 27,  73,  30,   0,   0, 107],
       [  0,  72,   0,   0, 115,  43],
       [ 27,   0,  30,   7, 107, 107],
       [ 23,  33,  27,   0,  30,  39],
       [ 25,  76,   0,   1,   0,   0],
       [ 23,  36,  27,   0,   0, 107],
       [  0,  43,   0,  61,   0, 107],
       [ 23,   0,  27,  66,  30,   0],
       [ 27,  76,  41,  38,  30,   0],
       [ 27,   0,  30, 11

## Check total of valid plugins created

In [38]:
import pandas as pd
filtered_ids = pd.DataFrame(ids[(ids != 0).all(axis=1)], columns=[f'plugin{i}' for i in range(1, 7)])

filtered_ids.sort_values(by=list(filtered_ids.columns.values))

,plugin1,plugin2,plugin3,plugin4,plugin5,plugin6
11,23,27,27,66,71,107
3,23,27,27,95,107,107
10,23,57,43,60,107,107
8,23,86,28,43,44,53
0,24,16,64,107,107,107
4,24,70,5,64,107,107
5,27,23,44,55,44,62
6,27,23,46,44,55,56
9,27,23,54,30,59,71
12,27,25,30,27,92,64


In [45]:
print('Valid patches (all six plugins has been sampled)')
filtered_ids.shape[0] / total_of_recommendations

Valid patches (all six plugins has been sampled)


0.14

## Check if all plugins are originals

In [50]:
from experiments.data.load_data_util import load_data, load_data_categories

data = load_data('../data')

ds1 = set(tuple(line) for line in data.values)
ds2 = set(tuple(line) for line in filtered_ids.values)

total_original = pd.DataFrame(list(ds2.difference(ds1)), columns=data.columns).sort_values(by=list(filtered_ids.columns.values)).shape[0]
f"There are {filtered_ids.shape[0] - total_original} non original plugins"

'There are 0 non original plugins'

## Selected plugins

In [61]:
categories = load_data_categories('../data')

patches_with_names = filtered_ids.copy()

for column in filtered_ids.columns:
    patches_with_names = patches_with_names.join(categories, on=column)
    del patches_with_names['category']
    del patches_with_names[column]
    patches_with_names = patches_with_names.rename(columns={'name': column})

patches_with_names

,plugin1,plugin2,plugin3,plugin4,plugin5,plugin6
0,RackComp,Cho+Dly,Arena,None,None,None
1,T Scream,GraphicEQ,ZNR,None,None,None
2,ZNR,Aco.Sim,GraphicEQ,FilterDly,None,None
3,Comp,ZNR,ZNR,BG CRUNCH,None,None
4,RackComp,PDL MnPit,FilterDly,Arena,None,None
5,ZNR,Comp,StereoCho,ModDelay,StereoCho,TiledRoom
6,ZNR,Comp,VinFLNGR,StereoCho,ModDelay,AnalogDly
7,PedalCry,Comp+AWah,StereoDly,StereoDly,ZNR,Delay
8,Comp,Z Clean,NoiseGate,VintageCE,StereoCho,Delay
9,ZNR,Comp,TapeEcho,GraphicEQ,DynaDelay,Booster


## Selected categories

In [63]:
patches_with_categories = filtered_ids.copy()

for column in filtered_ids.columns:
    patches_with_categories = patches_with_categories.join(categories, on=column)
    del patches_with_categories['name']
    del patches_with_categories[column]
    patches_with_categories = patches_with_categories.rename(columns={'category': column})

patches_with_categories

,plugin1,plugin2,plugin3,plugin4,plugin5,plugin6
0,Dynamics,Combination Fx,Reverb,None,None,None
1,Stomp,Filter_EQ,Dynamics,None,None,None
2,Dynamics,Stomp,Filter_EQ,Delay,None,None
3,Dynamics,Dynamics,Dynamics,Amp,None,None
4,Dynamics,Pedal Fx,Delay,Reverb,None,None
5,Dynamics,Dynamics,Modulation,Delay,Modulation,Reverb
6,Dynamics,Dynamics,Modulation,Modulation,Delay,Delay
7,Pedal Fx,Combination Fx,Delay,Delay,Dynamics,Delay
8,Dynamics,Amp,Dynamics,Modulation,Modulation,Delay
9,Dynamics,Dynamics,Delay,Filter_EQ,Delay,Stomp


## Check in database

There are common the use o boosters (id=71) in end of effects chain?

In [69]:
print(data[(data['plugin5'] == 71) | (data['plugin6'] == 71)].shape[0], 'cases')

print('Five examples')
data[(data['plugin5'] == 71) | (data['plugin6'] == 71)].head(5)

172 cases
Five examples


,,plugin1,plugin2,plugin3,plugin4,plugin5,plugin6
id,name,,,,,,
6584,BeatleSOFT,28,24,93,66,30,71
6730,Metalcore*,73,72,27,98,56,71
7445,CMNA*BOSS*,33,73,76,30,71,107
7699,Solo******,27,86,83,106,71,53
7908,Bass******,51,24,30,41,71,107


ZNR in end of chain?

In [78]:
data[(data['plugin5'] == 23) | (data['plugin6'] == 23)].head(5)

print('ZNR by position')
(data == 23).sum(axis=0)

ZNR by position


plugin1    377
plugin2    103
plugin3     16
plugin4      5
plugin5      2
plugin6      5
dtype: int64

In [83]:
data[(data['plugin3'] == 23)]

plugin1  plugin2  plugin3  plugin4  plugin5  plugin6
id    name                                                            
6342  Acoustic**       60       85       23       66       31       85
6903  Washed*Out      114       16       23      107      107      107
7314  410BlsMnDR       27       94       23      107      107      107
      Hot*Wet*Ld       30      112       23       59       27      107
9472  410BlsSolo       27       94       23       30       59       71
9672  OldSmith**       23       98       23      107      107      107
9913  Slash*****       28       68       23       73       99       59
10403 FENDER*TEL       27       44       23       46       30       60
      Vibrato***       27       85       23       48       30       56
      FLAMENCO**       27       85       23       30       66       60
      Trem*Clean       27       85       23       30       38       71
10430 410BlsSolo       27       94       23       30       56       71
      AWAH*SOLO*       27       33       23       72       30       59
      AWAH******       27       33       23       30       30       59
10580 CRY*FLAY**       27       36       23       30        4       56
10975 amp              31       92       23      107      107      107

Pitch shifter

In [85]:
test = data[(data['plugin2'] == 70)]

test = test.copy()

for column in test.columns:
    test = test.join(categories, on=column)
    del test['category']
    del test[column]
    test = test.rename(columns={'name': column})

test

,,plugin1,plugin2,plugin3,plugin4,plugin5,plugin6
id,name,,,,,,
6787,TomMorello,NoiseGate,PDL MnPit,OverDrive,BG CRUNCH,PhaseDly,Delay
6903,DZ*Bend***,NoiseGate,PDL MnPit,DZ DRIVE,EarlyRef,None,None
6955,TomMorello,NoiseGate,PDL MnPit,OverDrive,BG CRUNCH,GraphicEQ,Delay
7314,BendMod***,ZNR,PDL MnPit,Vibrato,BG DRIVE,Delay,None
9362,Psycho****,NoiseGate,PDL MnPit,GreatMuff,Phaser,MS CRUNCH,Hall
9472,Progress**,ZNR,PDL MnPit,Governor,MetalWRLD,PhaseDly,Booster
10215,Violin1,ZNR,PDL MnPit,SlowATTCK,TANGERINE,GraphicEQ,Dly+Rev
10403,Marooned**,ZNR,PDL MnPit,BG CRUNCH,DynaDelay,Detune,Booster
